In [7]:
"""
Static Search Strategy Function
"""

from typing import Dict, Any
import logging

logger = logging.getLogger(__name__)


def determine_okdb_search_strategy(vr_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Static function to determine OK DB search strategy based on entity type
    
    Args:
        vr_data: Parsed VR record containing entity type and identifiers (flat JSON structure)
    
    Returns:
        Dictionary with search parameters and strategy
    """
    try:
        # Validate input data
        if not vr_data or not isinstance(vr_data, dict):
            raise ValueError("VR data must be a non-empty dictionary")
        
        # Extract validation fields from flat structure
        entity_type = vr_data.get("entityTypeIco", "").strip()
        vr_id = vr_data.get("id")
        
        if not entity_type:
            raise ValueError("Entity type is required but missing")
        
        logger.info(f"Determining search strategy for VR ID: {vr_id}, Entity Type: {entity_type}")
        
        # ENT_ACTIVITY Logic
        if entity_type == "ENT_ACTIVITY":
            # Check for individual originKeyEid first
            individual_origin_key = vr_data.get("individualOriginKeyEid", "").strip()
            
            if individual_origin_key:
                logger.debug(f"VR ID {vr_id}: Using originKeyEid search for individual")
                return {
                    "search_method": "originkey_individual",
                    "search_params": {
                        "originKeyEid": individual_origin_key,
                        "searchType": "individual_by_originkey"
                    },
                    "entity_type": entity_type,
                }
            
            # Fallback to name-based search
            first_name = vr_data.get("firstName", "").strip()
            last_name = vr_data.get("lastName", "").strip()
            
            if not first_name and not last_name:
                raise ValueError("Individual must have either originKeyEid or name information")
            
            search_params = {"searchType": "individual_by_name"}
            
            # Add available fields
            if first_name:
                search_params["firstName"] = first_name
            if last_name:
                search_params["lastName"] = last_name
            
            workplace_name = vr_data.get("workplaceUsualName", "").strip()
            if workplace_name:
                search_params["workplace"] = workplace_name
            
            country = vr_data.get("country", "").strip()
            if country:
                search_params["country"] = country
            
            city = vr_data.get("city", "").strip()
            if city:
                search_params["city"] = city
            
            # Add specialty codes if available
            specialty_code1 = vr_data.get("specialityCode1", "").strip()
            if specialty_code1:
                search_params["specialityCode1"] = specialty_code1
            
            specialty_code2 = vr_data.get("specialityCode2", "").strip()
            if specialty_code2:
                search_params["specialityCode2"] = specialty_code2
            
            logger.debug(f"VR ID {vr_id}: Using name-based search with {len(search_params)} parameters")
            
            return {
                "search_method": "name_based",
                "search_params": search_params,
                "entity_type": entity_type,
            }
        
        # ENT_WORKPLACE Logic
        elif entity_type == "ENT_WORKPLACE":
            # Check for workplace originKeyEid first
            workplace_origin_key = vr_data.get("workplaceOriginKeyEid", "").strip()
            
            if workplace_origin_key:
                logger.debug(f"VR ID {vr_id}: Using originKeyEid search for workplace")
                return {
                    "search_method": "originkey_workplace",
                    "search_params": {
                        "originKeyEid": workplace_origin_key,
                        "searchType": "workplace_by_originkey"
                    },
                    "entity_type": entity_type,
                }
            
            # Fallback to workplace name-based search
            workplace_name = vr_data.get("workplaceUsualName", "").strip()
            
            if not workplace_name:
                raise ValueError("Workplace must have either originKeyEid or usualName")
            
            search_params = {
                "workplaceName": workplace_name,
                "searchType": "workplace_by_name"
            }
            
            # Add geographical context
            country = vr_data.get("country", "").strip()
            if country:
                search_params["country"] = country
            
            city = vr_data.get("city", "").strip()
            if city:
                search_params["city"] = city
            
            # Add workplace details
            type_code = vr_data.get("workplaceTypeCode", "").strip()
            if type_code:
                search_params["typeCode"] = type_code
            
            activity_location_code = vr_data.get("workplaceActivityLocationCode", "").strip()
            if activity_location_code:
                search_params["activityLocationCode"] = activity_location_code
            
            # Add address details
            long_label = vr_data.get("addressLongLabel", "").strip()
            if long_label:
                search_params["address"] = long_label
            
            postal_code = vr_data.get("postalCode", "").strip()
            if postal_code:
                search_params["postalCode"] = postal_code
            
            logger.debug(f"VR ID {vr_id}: Using workplace name-based search")
            
            return {
                "search_method": "workplace_based",
                "search_params": search_params,
                "entity_type": entity_type,
            }
        
        # Unknown entity type - fallback
        else:
            logger.warning(f"Unknown entity type: {entity_type}, using fallback strategy")
            
            # Try individual-based search as fallback
            first_name = vr_data.get("firstName", "").strip()
            last_name = vr_data.get("lastName", "").strip()
            
            if first_name or last_name:
                search_params = {"searchType": "fallback_search"}
                
                if first_name:
                    search_params["firstName"] = first_name
                if last_name:
                    search_params["lastName"] = last_name
                
                workplace_name = vr_data.get("workplaceUsualName", "").strip()
                if workplace_name:
                    search_params["workplace"] = workplace_name
                
                country = vr_data.get("country", "").strip()
                if country:
                    search_params["country"] = country
                
                return {
                    "search_method": "name_based",
                    "search_params": search_params,
                    "entity_type": f"UNKNOWN_{entity_type}",
                }
            
            # Last resort
            raise ValueError("Insufficient data to determine any viable search strategy")
    
    except Exception as e:
        logger.error(f"Error determining search strategy: {str(e)}")
        raise ValueError(f"Failed to determine search strategy: {str(e)}")

In [8]:
# VR Test Data Samples - Flat JSON Structure (as returned by VRDataClass)

# Sample 1 - ENT_ACTIVITY with individual originKeyEid
vr_sample_1 = {
    "refAreaEid": "RAR_ITALY",
    "id": 1019000131693245,
    "customerId": 2436,
    "externalId": "47064409",
    "customerRequestEid": "VVAIT797937-NA-NA-CGD",
    "vrTypeCode": "VMR",
    "countryCode": "IT",
    "entityTypeIco": "ENT_ACTIVITY",
    "integrationDate": "2025-06-02T18:06:29Z",
    "requestDate": "2025-05-30T12:00:00Z",
    "requestComment": "Claudia Gambini kol426 Presso l'ospedale di Velletri di Via Orti Ginnetti fa due giorni di visite diabetologiche e un giorno di diabete gestazionale-DCR000266334",
    "statusIco": "VAS_NOT_PROCESSED",
    "isForced": False,
    "businessStatusCode": "C",
    "statusDate": "2025-06-02T18:06:29Z",
    "requesterLastName": "dcr_integration_user",
    "slaRemainingNumDays": 1,
    "slaDate": "2025-06-03",
    "slaNumDays": 5,
    "individualWithoutActivityAccepted": False,
    # Individual fields
    "individualOriginKeyEid": "WITO43596247",
    "firstName": "PAOLO",
    "lastName": "CORVISIERI",
    "specialityCode1": "18",
    "specialityCode2": "29",
    # Workplace fields
    "workplaceOriginKeyEid": "WITO62731655",
    "workplaceUsualName": "DISTRETTO SANITARIO FIUMICINO",
    # Address fields
    "country": "IT",
    "city": "VELLETRI",
    "postalCity": "VELLETRI"
}

# Sample 2 - ENT_ACTIVITY without individual originKeyEid (name-based search)
vr_sample_2 = {
    "refAreaEid": "RAR_ITALY",
    "id": 1019000131693115,
    "customerId": 9271,
    "externalId": "47064466",
    "customerRequestEid": "VR44423202506021009443302",
    "vrTypeCode": "VMR",
    "countryCode": "IT",
    "entityTypeIco": "ENT_ACTIVITY",
    "integrationDate": "2025-06-02T12:10:46Z",
    "requestDate": "2025-06-02T12:10:46Z",
    "statusIco": "VAS_NOT_PROCESSED",
    "isForced": False,
    "businessStatusCode": "C",
    "statusDate": "2025-06-02T12:10:46Z",
    "requesterId": "Q053K000000ESbSRQA5",
    "requesterLastName": "Rocco Maiorano",
    "slaRemainingNumDays": 1,
    "slaDate": "2025-06-09",
    "slaNumDays": 5,
    "individualWithoutActivityAccepted": False,
    # Individual fields (NO originKeyEid - triggers name-based search)
    "firstName": "GAETANO",
    "lastName": "CANALE",
    "specialityCode1": "08",
    "specialityCode2": "29",
    # Workplace fields
    "workplaceOriginKeyEid": "WITE000120355",
    "workplaceUsualName": "DISTRETTO SANITARIO FIUMICINO",
    "workplaceTypeCode": "DST",
    "workplaceActivityLocationCode": "A",
    "workplaceManagerUsualName": "ASL ROMA 3",
    "workplaceTelephone": "06-56483822",
    # Address fields
    "country": "IT",
    "addressLongLabel": "via coni zugna 173",
    "city": "FIUMICINO",
    "postalCode": "00054",
    "postalCity": "FIUMICINO"
}

# Sample 3 - ENT_WORKPLACE with workplace originKeyEid
vr_sample_3 = {
    "refAreaEid": "RAR_ITALY",
    "id": 1019000131743075,
    "customerId": 6675,
    "externalId": "47108375",
    "customerRequestEid": "VR14647202506041502203044",
    "vrTypeCode": "VMR",
    "countryCode": "IT",
    "entityTypeIco": "ENT_WORKPLACE",
    "integrationDate": "2025-06-04T17:05:54Z",
    "requestDate": "2025-06-04T17:05:54Z",
    "statusIco": "VAS_NOT_PROCESSED",
    "isForced": False,
    "businessStatusCode": "C",
    "statusDate": "2025-06-04T17:05:54Z",
    "requesterId": "0055000002NtDyTAK",
    "requesterLastName": "Cristian Lucio Gautiero",
    "slaRemainingNumDays": 3,
    "slaDate": "2025-06-11",
    "slaNumDays": 5,
    "individualWithoutActivityAccepted": False,
    # Workplace fields
    "workplaceOriginKeyEid": "WITE09992099",
    "workplaceUsualName": "SANITARIA ORTOPEDIA SANTALI",
    "workplaceTypeCode": "SAN",
    "workplaceActivityLocationCode": "T",
    # Address fields
    "country": "IT",
    "addressLongLabel": "VIA MORIGGIA PELASCINI 3",
    "city": "GRAVEDONA ED UNITI",
    "postalCode": "22015",
    "postalCity": "GRAVEDONA ED UNITI"
}

# Sample 4 - ENT_WORKPLACE without originKeyEid (workplace name-based search)
vr_sample_4 = {
    "refAreaEid": "RAR_ITALY", 
    "id": 1019000131693050,
    "customerId": 9102,
    "externalId": "47064455",
    "customerRequestEid": "VR09920250602085629202",
    "vrTypeCode": "VMR",
    "countryCode": "IT",
    "entityTypeIco": "ENT_WORKPLACE",
    "integrationDate": "2025-06-02T10:57:42Z",
    "requestDate": "2025-06-02T10:57:42Z", 
    "statusIco": "VAS_NOT_PROCESSED",
    "isForced": False,
    "businessStatusCode": "C",
    "statusDate": "2025-06-02T10:57:42Z",
    "requesterId": "005KA0000000xmvYAA",
    "requesterLastName": "Vita Fedele",
    "slaRemainingNumDays": 1,
    "slaDate": "2025-06-09",
    "slaNumDays": 5,
    "individualWithoutActivityAccepted": False,
    # Workplace fields (NO originKeyEid - triggers workplace name-based search)
    "workplaceUsualName": "Marco Oradei Medical Practice",
    "workplaceSpecialityCode1": "33",
    # Address fields
    "country": "IT",
    "addressLongLabel": "Largo F. Vito 1, 00168 Roma",
    "city": "Roma",
    "postalCode": "00054",
    "postalCity": "Roma"
}

# # Test cases for validation
# test_cases = [
#     {
#         "name": "ENT_ACTIVITY with individual originKeyEid",
#         "data": vr_sample_1,
#         "expected_method": "originkey_individual",
#         "expected_search_type": "individual_by_originkey"
#     },
#     {
#         "name": "ENT_ACTIVITY without individual originKeyEid", 
#         "data": vr_sample_2,
#         "expected_method": "name_based",
#         "expected_search_type": "individual_by_name"
#     },
#     {
#         "name": "ENT_WORKPLACE with workplace originKeyEid",
#         "data": vr_sample_3,
#         "expected_method": "originkey_workplace",
#         "expected_search_type": "workplace_by_originkey"
#     },
#     {
#         "name": "ENT_WORKPLACE without workplace originKeyEid",
#         "data": vr_sample_4,
#         "expected_method": "workplace_based",
#         "expected_search_type": "workplace_by_name"
#     }
# ]

# # Test function
# def test_search_strategy():
#     """Test the search strategy function with all scenarios"""
#     from utils.search_strategy import determine_okdb_search_strategy
    
#     print("=== Testing VR Search Strategy Function ===\n")
    
#     for i, test_case in enumerate(test_cases, 1):
#         print(f"Test {i}: {test_case['name']}")
#         print(f"VR ID: {test_case['data']['id']}")
#         print(f"Entity Type: {test_case['data']['entityTypeIco']}")
        
#         try:
#             result = determine_okdb_search_strategy(test_case['data'])
            
#             print(f"✅ SUCCESS")
#             print(f"   Search Method: {result['search_method']}")
#             print(f"   Search Type: {result['search_params']['searchType']}")
#             print(f"   Confidence: {result['confidence_level']}")
#             print(f"   Parameters: {len(result['search_params'])} total")
            
#             # Validate expected results
#             if result['search_method'] == test_case['expected_method']:
#                 print(f"   ✅ Method matches expected: {test_case['expected_method']}")
#             else:
#                 print(f"   ❌ Method mismatch! Expected: {test_case['expected_method']}, Got: {result['search_method']}")
            
#             if result['search_params']['searchType'] == test_case['expected_search_type']:
#                 print(f"   ✅ Search type matches expected: {test_case['expected_search_type']}")
#             else:
#                 print(f"   ❌ Search type mismatch! Expected: {test_case['expected_search_type']}, Got: {result['search_params']['searchType']}")
            
#             # Show key parameters
#             key_params = {k: v for k, v in result['search_params'].items() if k != 'searchType'}
#             if key_params:
#                 print(f"   Key Parameters: {key_params}")
                
#         except Exception as e:
#             print(f"❌ ERROR: {str(e)}")
        
#         print("-" * 60)
    
#     print("=== Test Complete ===")

# if __name__ == "__main__":
#     test_search_strategy()

In [10]:
determine_okdb_search_strategy(vr_sample_1)

{'search_method': 'originkey_individual',
 'search_params': {'originKeyEid': 'WITO43596247',
  'searchType': 'individual_by_originkey'},
 'entity_type': 'ENT_ACTIVITY'}

In [11]:
determine_okdb_search_strategy(vr_sample_2)

{'search_method': 'name_based',
 'search_params': {'searchType': 'individual_by_name',
  'firstName': 'GAETANO',
  'lastName': 'CANALE',
  'workplace': 'DISTRETTO SANITARIO FIUMICINO',
  'country': 'IT',
  'city': 'FIUMICINO',
  'specialityCode1': '08',
  'specialityCode2': '29'},
 'entity_type': 'ENT_ACTIVITY'}

In [12]:
determine_okdb_search_strategy(vr_sample_3)

{'search_method': 'originkey_workplace',
 'search_params': {'originKeyEid': 'WITE09992099',
  'searchType': 'workplace_by_originkey'},
 'entity_type': 'ENT_WORKPLACE'}

In [13]:
determine_okdb_search_strategy(vr_sample_4)

{'search_method': 'workplace_based',
 'search_params': {'workplaceName': 'Marco Oradei Medical Practice',
  'searchType': 'workplace_by_name',
  'country': 'IT',
  'city': 'Roma',
  'address': 'Largo F. Vito 1, 00168 Roma',
  'postalCode': '00054'},
 'entity_type': 'ENT_WORKPLACE'}